In [4]:
# To find ECD Control

import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')


In [6]:
import numpy as np
from qutip import *
from ECD_batch_optimizerV1 import BatchOptimizer
import matplotlib.pyplot as plt

C:\Users\Eesh Gupta\Documents\RU Research\Chakram\Multimode-Conditional-Displacements\hpc_runs\single_mode_ecd\Varying Time Lengths\ECD_batch_optimizerV1.py:153: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (filename is not None and filename is not "")
C:\Users\Eesh Gupta\Documents\RU Research\Chakram\Multimode-Conditional-Displacements\hpc_runs\single_mode_ecd\Varying Time Lengths\ECD_batch_optimizerV1.py:658: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  time_per_epoch = elapsed_time_s / epoch if epoch is not 0 else 0.0



Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
#The target oscillator state.
N =2
Fock = 1
psi_i = tensor(basis(2,0), basis(N,0)) # initial state
psi_t = tensor(basis(2,0), basis(N,Fock)) #target state

In [9]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 4, #circuit depth
'N_multistart' : 200, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 200, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.995, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [psi_i], #qubit tensor oscillator, start in |g> |0>
'target_states' : [psi_t], #end in |e> |target>.
'name' : 'Fock %d' % Fock, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

#### Filename for angles to be stored

In [10]:
def main(layers, fi):
    '''
    Vary layers, get fidelity
    '''
    for layer in layers: 
        #optimizer
        opt_params['N_blocks'] = layers
        opt = BatchOptimizer(**opt_params)
        opt.optimize()
        BO_fid = opt.best_fidelity()
        angles_filename = 'test_opt_angles.txt'
        opt.save_angles(filename = angles_filename)
        
        #pulses
        pulse_sim = ecd_pulse_single_mode(param_file = angles_filename,
                                 kappa = 1/(2e+6)) 
        pulse_sim.get_pulses()
        
        pulse_time = len(pulse_sim.alpha)
        
        df_new = pd.DataFrame([[layer, pulse_time, BO_fid]], columns=columns)
        if type(df) ==None : 
            df = df_new
        else:
            df = df.append(df_new, ignore_index = True)
        df.to_csv(fname, index=False)

None
Tensor("alphas:0", shape=(1, 200), dtype=complex64)
Tensor("alphas:0", shape=(4, 200), dtype=complex64)
optimization_type: state transfer
N_multistart: 200
N_blocks: 4
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 200
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
user_angles: None
N_cav: 2
filename: Fock 1.h5

Best circuit parameters found:
betas:         [-1.78562+0.53807j -1.47755+0.28664j -0.51977+0.34508j -0.93078+2.73289j]
alphas:        [0.+0.j]
phis (deg):    [   0.      -157.34834 -163.97528 -158.28035]
thetas (deg):  [113.32277  52.1676  -11.82537 118.69343]
Max Fidelity:  0.918585




In [11]:
#run optimizer.
opt.optimize()

Start time: 2022-10-21 08:37:41
 Epoch: 0 / 200 Max Fid: 0.918585 Avg Fid: 0.275893 Max dFid: 0.000000 Avg dFid: 0.000000 Elapsed time: 0:00:00.003989 Remaing time: 0:00:00

KeyboardInterrupt: 